In [5]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import sys
import json
import numpy as np
sys.path.append("..")
from data.loaders import load_capas_from_jsons

In [3]:
base_dir = r"C:\Users\stav\data\whodis\parsed\CAPAs"
cat_dfs= load_capas_from_jsons(base_dir)
capas_df = cat_dfs['capas']
mbcs_df = cat_dfs['mbc']

Loading json files: 100%|█████████████████████████████████████████████████████████| 2148/2148 [00:02<00:00, 815.89it/s]


In [25]:
def _prepare_dfs_for_model(cats_df, column):
    cats_df['dummy'] = 1
    features_df = pd.pivot_table(cats_df, values='dummy', index=['uid'], columns=[column], aggfunc=np.sum, fill_value=0)
    features_df = (features_df>=1).astype(int)
    features_df['label'] = capas_df.groupby('uid').label.first()
    return features_df
    

In [26]:
# Prepare 
capas_features_df = _prepare_dfs_for_model(capas_df, column='rule')
mbcs_features_df = _prepare_dfs_for_model(mbcs_df,column='objective')


In [27]:
capas_features_df[np.setdiff1d(capas_features_df.columns, 'label')].sum(1)

uid
0024d27614933d13f0ba3c380e28558d9470861bcced22ff2be822ec098a8646    14
003a903cbab3e91ef22602624645342e81b30ec6263f1e62165abd479c48942e     7
0041d82cc2a5bb2f447f9c40989292675232cb8404bbfa751c9b961e2f6149c7    15
004ede55a972e10d9a21bcf338b4907d6eed65bf5ad6abbbd5aec7d8484bdedf     7
00b4e13fd8eaae74e4244919a2301ee56599a6f3b4863a7f6a2a40564c65409d    26
                                                                    ..
ffce2c84295a89f0fefd475ad51f085005cfdb6456d3d7baf4e4be5076e9dfad     6
ffcf325fdc1ac66c9dbac40a97ba7c8f082641e14fc9d1d2343792e56ee8b250    26
ffd6bdff8e65b8fd44ea57a068a240b96b5b5598cd6aef89a8683bbed5babb48    26
ffdf9d72522eb970300f2229b500ad0bdcc3d6d4a902ed608a1032b7bea59ca5    25
fff3d9dc16bf3a8c3181ed99c6e93577053db0c13b1400cc7f19440c69f12930    26
Length: 2147, dtype: int64

In [22]:
from catboost import CatBoost, CatBoostRegressor, Pool
df_ = df.sample(frac=1., random_state=0)
df_train = df_.iloc[: 2 * len(df) // 3]
df_test = df_.iloc[2 * len(df) // 3 :]
train_pool = Pool(df_train.drop(['price_usd'], 1), 
                  label=df_train.price_usd,
                  cat_features=categorical_features_names)
test_pool = Pool(df_test.drop(['price_usd'], 1),
                 label=df_test.price_usd,
                 cat_features=categorical_features_names)

array(['accept command line arguments', 'access PEB ldr_data',
       'allocate RWX memory', 'allocate thread local storage',
       'authenticate HMAC', 'bypass Windows File Protection',
       'capture screenshot', 'change the wallpaper', 'check OS version',
       'check for PEB BeingDebugged flag',
       'check for PEB NtGlobalFlag flag',
       'check for software breakpoints',
       'check for time delay via GetTickCount',
       'check for unmoving mouse cursor', 'check if directory exists',
       'check if file exists', 'compile .NET assembly',
       'compiled to the .NET platform', 'compiled with Borland Delphi',
       'compress data via WinAPI', 'contain a resource (.rsrc) section',
       'contain a thread local storage (.tls) section',
       'contain an embedded PE file', 'contain obfuscated stackstrings',
       'contains PDB path', 'copy file', 'create HTTP request',
       'create UDP socket', 'create directory', 'create mutex',
       'create or open registry key'